# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [3]:
# import numpy and pandas
import pandas as pd
import numpy as np
from scipy.stats import trim_mean, mode, skew, gaussian_kde, pearsonr, spearmanr, beta

In [5]:
# import numpy and pandas
import pandas as pd
import numpy as np
from scipy.stats import trim_mean, mode, skew, gaussian_kde, pearsonr, spearmanr, beta
from statsmodels.stats.weightstats import ztest as ztest

from scipy.stats import ttest_ind, norm, t
from scipy.stats import f_oneway
from scipy.stats import sem

# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents

In [6]:
# Run this code:
salaries = pd.read_csv('../data/Current_Employee_Names__Salaries__and_Position_Titles.csv')

Examine the `salaries` dataset using the `head` function below.

In [7]:
# Your code here
salaries.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


# Challenge 2
This is a placeholder to make the AI corrector be able to find the correct exercise for feedback

# Challenge 3 - Constructing Confidence Intervals

We will test whether the hourly wage of all hourly workers is significantly different from $30/hr.

In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. Is $30/hr within that interval?

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [8]:
# Your code here
import numpy as np
import pandas as pd
from scipy.stats import sem, t

In [11]:
# 1) Filter down to only the hourly workers
hourly = salaries[salaries['Salary or Hourly'] == 'Hourly']


In [13]:
print(salaries.columns.tolist())

['Name', 'Job Titles', 'Department', 'Full or Part-Time', 'Salary or Hourly', 'Typical Hours', 'Annual Salary', 'Hourly Rate']


In [ ]:
# 2) Grab the hourly rate column and drop any NaNs
hourly_rates = hourly['Hourly Rate'].dropna()

In [16]:
# 3) Compute sample statistics
n       = len(hourly_rates)               # sample size
dfree   = n - 1                           # degrees of freedom
mean_hr = hourly_rates.mean()             # sample mean hourly rate
se_hr   = sem(hourly_rates, ddof=1)       # standard error (ddof=1)

In [28]:
# 4) Build the 95% CI via Student’s t
ci_low, ci_high = t.interval(0.95, dfree,
                             loc=mean_hr,
                             scale=se_hr)

print(f"Sample size (hourly): {n}")
print(f"Mean hourly rate:       ${mean_hr:.2f}")
print(f"95% CI:                 (${ci_low:.2f}, ${ci_high:.2f})")

Sample size (hourly): 8022
Mean hourly rate:       $32.79
95% CI:                 ($32.52, $33.05)


In [29]:
from statsmodels.stats.weightstats import DescrStatsW

ds = DescrStatsW(hourly_rates)
ci_low, ci_high = ds.tconfint_mean(alpha=0.05)

print(f"95% CI via statsmodels: ({ci_low:.2f}, {ci_high:.2f})")

95% CI via statsmodels: (32.52, 33.05)


In [30]:
# 5) Test if $30/hr lies inside
if ci_low <= 30 <= ci_high:
    print("$30/hr **is** within the 95% confidence interval.")
else:
    print("$30/hr **is NOT** within the 95% confidence interval.")

$30/hr **is NOT** within the 95% confidence interval.


This is fine if we have thousands of worker data. But what if we have only 100 workers data?

Sample 100 workers and re-construct the 95% confidence interval. Is the interval wider of narrower? And why?
Do you still encapsulate the $30/hr mark in this case?

In [32]:
#Draw a random sample of 100
sample100 = hourly_rates.sample(n=100)
n_samp    = len(sample100)
df_s      = n_samp - 1
mean_s    = sample100.mean()
se_s      = sem(sample100, ddof=1)
ci_s_low, ci_s_high = t.interval(0.95, df_s, loc=mean_s, scale=se_s)
width_s   = ci_s_high - ci_s_low

print(f"\nSample of 100 → 95% CI = (${ci_s_low:.2f}, ${ci_s_high:.2f}), width = {width_s:.2f}")


Sample of 100 → 95% CI = ($29.60, $33.90), width = 4.30


Looks like the interval is wider due to selecting a sample of only 100 workers instead the complete population.
The $30/hr mark in this case has fallen putise the CI with 95% of confidence.